In [1]:
import os

# os.environ["WANDB_API_KEY"] = '+++++++++++'  # 将引号内的+替换成自己在wandb上的一串值
# os.environ["WANDB_MODE"] = "offline"  # 离线  （此行代码不用修改）

import json

import pandas as pd
import torch
from datasets import Dataset
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from swanlab.integration.huggingface import SwanLabCallback
from transformers import DataCollatorForSeq2Seq, Trainer
import os

/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
glm4_model_path = '/home/LLM_para/para_glm4'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# 加载model和tokenizer
tokenizer = AutoTokenizer.from_pretrained(glm4_model_path, use_fast=False,
                                          trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(glm4_model_path,
                                             #quantization_config=bnb_config,
                                             device_map="auto", torch_dtype=torch.bfloat16,
#                                              attn_implementation="flash_attention_2",
                                             trust_remote_code=True)
model.config.use_cache = False

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Loading checkpoint shards: 100%|████████████████████████████████████████████████████████| 10/10 [00:18<00:00,  1.85s/it]


In [4]:
model.enable_input_require_grads()  #  开启梯度检查点
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
#结束标识符
print(tokenizer.eos_token)
tokenizer.encode('<|endoftext|>')

<|endoftext|>


[151331, 151333, 151329]

In [5]:
# 拆解训练集
# 提供每次训练token数
from tqdm import tqdm
import copy 
import re
#获取表数据
rec_tablename_pattern = r"CREATE TABLE `([^`]+)`"
# table_name = re.search(rec_tablename_pattern, item).group(1).lower()

df = pd.read_csv("mydataset_new/table_schema_Reference_cropped.csv",encoding='utf-8')

table_name_list = []
for index, row in tqdm(df.iterrows(), total=len(df)):
    reference_tables = row['Reference_group']
    table_name =  re.search(rec_tablename_pattern, reference_tables).group(1)
    if table_name is None:
        print("error")
    table_name_list.append(table_name)


df = pd.read_csv("mydataset_new/table_schema_noReference_cropped.csv",encoding='utf-8')
for index, row in tqdm(df.iterrows(), total=len(df)):
    reference_tables = row['noReference_group']
    table_name = re.search(rec_tablename_pattern, reference_tables).group(1)
    if table_name is None:
        print("error")
    table_name_list.append(table_name)
# print(table_name_list)
# print(len(table_name_list))
table_name_str = ", ".join(table_name_list)
schema_str = "[" + table_name_str + "]"
print(table_name_str)
check_token = tokenizer(f"{schema_str}", add_special_tokens=False)    
print(len(check_token["input_ids"]))
max_token = 900

100%|████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 14158.78it/s]

Manufacturers, Products, Student, Plays_Games, SportsInfo, actor, entrepreneur, people, Ref_Hotel_Star_Ratings, Ref_Attraction_Types, Hotels, Tourist_Attractions, Street_Markets, Shops, Museums, Royal_Family, Theme_Parks, Visits, Photos, Staff, Tourist_Attraction_Features, wrestler, Elimination, business, category, checkin, neighbourhood, review, tip, Ref_Detention_Type, Ref_Incident_Type, Addresses, Students, Teachers, Assessment_Notes, Behavior_Incident, Detention, Student_Addresses, Students_in_Detention, film, film_market_estimation, Catalogs, Catalog_Structure, Catalog_Contents, Catalog_Contents_Additional_Attributes, routes, airports, stadium, game, injury_accident, Physician, Department, Affiliated_With, Trained_In, Patient, Nurse, Appointment, Prescribes, Block, Room, On_Call, Stay, Undergoes, buildings, Office_locations, region, party, member, party_events, Web_client_accelerator, accelerator_compatible_browser, Investors, Lots, Transactions, Sales, Purchases, Transactions_Lot

In [6]:
def fill_dataset_template(schema_pad,question_pad,answer_pad):
     message = {
                "instruction": f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{schema_pad}",
                "input": f"{question_pad}",
                "output": f"```Reference Table\n-- Tables: {answer_pad};\n```",
            }
     return message
#  #None#

In [8]:
df = pd.read_csv("mydataset_new/full_finetuning_dataset_cropped.csv",encoding="utf-8")
outer_index = 0
message_group = []
for i in range(len(df['db_id'])):
    question = df['question'][i]
    query = df['query'][i]
    correct_table = df['correct_tables'][i]
    dict_target_table_slice = {}
    message = fill_dataset_template(schema_str,question,correct_table)     
    message_group.append(message)
        
with open("T2Q_GLM4_SFT_train_tableName_myidea.jsonl", "w", encoding="utf-8") as file:
    for message in message_group:
        file.write(json.dumps(message, ensure_ascii=False) + "\n")

print(len(message_group))
#     判断

        
#     table_name = re.search(r"CREATE TABLE `([^`]+)`", table).group(1)
#     print(correct_table_list)
    #      schema_slicing
#     db_id = df['db_id'][i].replace(" ", "")
#     message = {
#         "instruction": f" I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. \n##instruction:{database_schema}",
#         "input": f"{question}",
#         "output": f"```Reference Table\n-- Tables: {correct_tables};\n```",
#     }
#     messages.append(message)
#     # print(message["instruction"])
#     # print(message["input"])
#     # print(message["output"])
#     print("++++++++++++++++++++++++++++++++++++++++++++++++++++")
#     if max_length < len(message["instruction"]) + len(message["input"]) + len(message["output"]):
#         max_length = len(message["instruction"]) + len(message["input"]) + len(message["output"])
#         max_ins = message["instruction"]
#         max_in = message["input"]
#         max_out = message["output"]

# with open("./My_idea_dataset/T2Q_GLM4_SFT_train_table.jsonl", "w", encoding="utf-8") as file:
#     for message in messages:
#         file.write(json.dumps(message, ensure_ascii=False) + "\n")

1736


In [9]:
# 生成训练集
train_path = "T2Q_GLM4_SFT_train_tableName_myidea.jsonl"
max_len = 0
count = 0
with open(train_path, "r") as file:
        for line in file:
            count+=1
            # 解析每一行的json数据
            example = json.loads(line)
            if count == 1:
                print(f"<|system|>\n {example['instruction']}.<|endoftext|>\n<|user|>\n{example['input']}<|endoftext|>\n<|assistant|>\n")
            print(f"index:---------{count}")
            instruction = tokenizer(
        f"<|system|>\n {example['instruction']}.<|endoftext|>\n<|user|>\n{example['input']}<|endoftext|>\n<|assistant|>\n",
        add_special_tokens=False, )
            response = tokenizer(f"{example['output']}<|endoftext|>\n", add_special_tokens=False)
            input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
            attention_mask = (
            instruction["attention_mask"] + response["attention_mask"] + [1])
            labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
            if len(input_ids) > max_len:
                max_len = len(input_ids)
print("max_len:",max_len)   
#1860

<|system|>
  I want you to act as a relation extraction robot for a sample SQL table. You only need to return the tables related to the user's input question. Below are instructions describing the relationship between tables. Please write a response that appropriately completes the request. 
##instruction:[Manufacturers, Products, Student, Plays_Games, SportsInfo, actor, entrepreneur, people, Ref_Hotel_Star_Ratings, Ref_Attraction_Types, Hotels, Tourist_Attractions, Street_Markets, Shops, Museums, Royal_Family, Theme_Parks, Visits, Photos, Staff, Tourist_Attraction_Features, wrestler, Elimination, business, category, checkin, neighbourhood, review, tip, Ref_Detention_Type, Ref_Incident_Type, Addresses, Students, Teachers, Assessment_Notes, Behavior_Incident, Detention, Student_Addresses, Students_in_Detention, film, film_market_estimation, Catalogs, Catalog_Structure, Catalog_Contents, Catalog_Contents_Additional_Attributes, routes, airports, stadium, game, injury_accident, Physician, 

index:---------344
index:---------345
index:---------346
index:---------347
index:---------348
index:---------349
index:---------350
index:---------351
index:---------352
index:---------353
index:---------354
index:---------355
index:---------356
index:---------357
index:---------358
index:---------359
index:---------360
index:---------361
index:---------362
index:---------363
index:---------364
index:---------365
index:---------366
index:---------367
index:---------368
index:---------369
index:---------370
index:---------371
index:---------372
index:---------373
index:---------374
index:---------375
index:---------376
index:---------377
index:---------378
index:---------379
index:---------380
index:---------381
index:---------382
index:---------383
index:---------384
index:---------385
index:---------386
index:---------387
index:---------388
index:---------389
index:---------390
index:---------391
index:---------392
index:---------393
index:---------394
index:---------395
index:------

index:---------823
index:---------824
index:---------825
index:---------826
index:---------827
index:---------828
index:---------829
index:---------830
index:---------831
index:---------832
index:---------833
index:---------834
index:---------835
index:---------836
index:---------837
index:---------838
index:---------839
index:---------840
index:---------841
index:---------842
index:---------843
index:---------844
index:---------845
index:---------846
index:---------847
index:---------848
index:---------849
index:---------850
index:---------851
index:---------852
index:---------853
index:---------854
index:---------855
index:---------856
index:---------857
index:---------858
index:---------859
index:---------860
index:---------861
index:---------862
index:---------863
index:---------864
index:---------865
index:---------866
index:---------867
index:---------868
index:---------869
index:---------870
index:---------871
index:---------872
index:---------873
index:---------874
index:------

index:---------1308
index:---------1309
index:---------1310
index:---------1311
index:---------1312
index:---------1313
index:---------1314
index:---------1315
index:---------1316
index:---------1317
index:---------1318
index:---------1319
index:---------1320
index:---------1321
index:---------1322
index:---------1323
index:---------1324
index:---------1325
index:---------1326
index:---------1327
index:---------1328
index:---------1329
index:---------1330
index:---------1331
index:---------1332
index:---------1333
index:---------1334
index:---------1335
index:---------1336
index:---------1337
index:---------1338
index:---------1339
index:---------1340
index:---------1341
index:---------1342
index:---------1343
index:---------1344
index:---------1345
index:---------1346
index:---------1347
index:---------1348
index:---------1349
index:---------1350
index:---------1351
index:---------1352
index:---------1353
index:---------1354
index:---------1355
index:---------1356
index:---------1357


In [10]:
#  和下面的max_len 作一个合并
#  max_seq_length = 2100
def process_func(example):
    """
    将数据集进行预处理
    """
    MAX_LENGTH = max_token 
    input_ids, attention_mask, labels = [], [], []
    instruction = tokenizer(
        f"<|system|>\n {example['instruction']}.<|endoftext|>\n<|user|>\n{example['input']}<|endoftext|>\n<|assistant|>\n",
        add_special_tokens=False, )
   
    response = tokenizer(f"{example['output']}", add_special_tokens=False)
    input_ids = instruction["input_ids"] + response["input_ids"] + [tokenizer.pad_token_id]
    attention_mask = (
        instruction["attention_mask"] + response["attention_mask"] + [1]
    )
    labels = [-100] * len(instruction["input_ids"]) + response["input_ids"] + [tokenizer.pad_token_id]
    if len(input_ids) > MAX_LENGTH:  # 做一个截断
        input_ids = input_ids[:MAX_LENGTH]
        attention_mask = attention_mask[:MAX_LENGTH]
        labels = labels[:MAX_LENGTH]
    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

In [11]:
#  set dataset
# T2Q_GLM4_SFT_train_tableName_myidea.jsonl
# val_path = "/home/code/chat_SQL/Dataset/GLM4/ENG_ori/My_idea_dataset/T2Q_GLM4_SFT_val_table.jsonl"

train_df = pd.read_json(train_path, lines=True)
train_ds = Dataset.from_pandas(train_df)
train_dataset = train_ds.map(process_func, remove_columns=train_ds.column_names)


# val_df = pd.read_json(val_path, lines=True)
# val_ds = Dataset.from_pandas(val_df)
# val_dataset = val_ds.map(process_func, remove_columns=val_ds.column_names)

Map: 100%|██████████████████████████████████████████████████████████████████| 1736/1736 [00:03<00:00, 510.81 examples/s]


In [12]:
#  train_config
#  示例里 r 为 8
lora_r = 64
lora_alpha = 32
lora_dropout = 0.1
output_dir = "./GLM4_SFT_TableOnly_Eng"
num_train_epochs = 6
bf16 = True
overwrite_output_dir = True
per_device_train_batch_size = 4
per_device_eval_batch_size = 20
gradient_accumulation_steps = 16
gradient_checkpointing = True
evaluation_strategy = "steps"
learning_rate = 5e-5
weight_decay = 0.01
lr_scheduler_type = "cosine"
warmup_ratio = 0.01
max_grad_norm = 0.3
group_by_length = True
auto_find_batch_size = False
save_steps = 50
logging_steps = 10
load_best_model_at_end= False
packing = False
save_total_limit=4
neftune_noise_alpha=5
# report_to="wandb"
max_seq_length = max_token

In [13]:
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    target_modules=[
       "query_key_value", "dense", "dense_h_to_4h", "activation_func", "dense_4h_to_h"
    ],
    task_type=TaskType.CAUSAL_LM,
)

In [14]:
swanlab_callback = SwanLabCallback(
    project="ENG_SFT_T2QTabNameOnly",
    experiment_name="GLM4-9B-2epo",
    description="使用智谱GLM4-9B-Chat模型微调MY_spider改数据集。",
    config={
        "model": "para_glm4",
        "dataset": "My_idea_T2Q_GLM4_SFT_train_table.jsonl",
    },
)

In [15]:
training_arguments = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=overwrite_output_dir,
    num_train_epochs=num_train_epochs,
    load_best_model_at_end=load_best_model_at_end,
    per_device_train_batch_size=per_device_train_batch_size,
#     evaluation_strategy=evaluation_strategy,
    max_grad_norm = max_grad_norm,
    auto_find_batch_size = auto_find_batch_size,
    save_total_limit = save_total_limit,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    bf16=bf16,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="none",
    neftune_noise_alpha= neftune_noise_alpha
)

In [16]:
# response_template = "### Response:"
# collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
# collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True)
# trainer = SFTTrainer(
#     model=model,
#     train_dataset=dataset['train'],
#     eval_dataset=dataset['validation'],
#     peft_config=peft_config,
#     data_collator=collator,
#     args=training_arguments,
#     max_seq_length=max_seq_length,
#     packing=packing
# )
model = get_peft_model(model, peft_config)
trainer = Trainer(
    model=model,
    args=training_arguments,
    train_dataset=train_dataset,
#     eval_dataset= val_dataset,
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
    callbacks=[swanlab_callback],
)

Detected kernel version 5.4.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


In [17]:
# import os
# os.environ['CUDA_HOME'] = '/usr/local/cuda'
trainer.train()

swanlab: Tracking run with swanlab version 0.3.13                                  
swanlab: Run data will be saved locally in /home/code/chat_SQL/exp4_main/myidea_verify/swanlog/run-20240910_135127-a3b1799d
swanlab: 👋 Hi winhong, welcome to swanlab!
swanlab: Syncing run GLM4-9B-2epo_Sep10_13-51-27 to the cloud
swanlab: 🌟 Run `swanlab watch -l /home/code/chat_SQL/exp4_main/myidea_verify/swanlog` to view SwanLab Experiment Dashboard locally
swanlab: 🏠 View project at https://swanlab.cn/@winhong/ENG_SFT_T2QTabNameOnly
swanlab: 🚀 View run at https://swanlab.cn/@winhong/ENG_SFT_T2QTabNameOnly/runs/2shjjxid9cko0c586zspy


Step,Training Loss
10,2.797700
20,0.269500
30,0.148900
40,0.108600
50,0.090200
60,0.077700
70,0.060900
80,0.051300
90,0.041600
100,0.033200


/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_glm4 - will assume that the vocabulary was not modified.
  warnings.warn(
/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_glm4 - will assume that the vocabulary was not modified.
  warnings.warn(
/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_glm4 - will assume that the vocabulary was not modified.
  warnings.warn(


TrainOutput(global_step=162, training_loss=0.23540079391297, metrics={'train_runtime': 1905.5502, 'train_samples_per_second': 5.466, 'train_steps_per_second': 0.085, 'total_flos': 3.369377407901368e+17, 'train_loss': 0.23540079391297, 'epoch': 5.972350230414746})

In [18]:
output_dir = os.path.join("./", "final_checkpoint_GLM4_TNonly-E_SFT_myidea")
trainer.model.save_pretrained(output_dir)

/home/vipuser/anaconda3/envs/GLM_4/lib/python3.10/site-packages/peft/utils/save_and_load.py:154: UserWarning: Could not find a config file in /home/LLM_para/para_glm4 - will assume that the vocabulary was not modified.
  warnings.warn(


In [ ]:
def predict(messages, model, tokenizer):
#     device = "cuda"
    text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    model_inputs = tokenizer([text], return_tensors="pt").to(device)

    generated_ids = model.generate(
        model_inputs.input_ids,
        max_new_tokens=512
    )
    generated_ids = [
        output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
    ]
    
    response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
    
    print(response)
     
    return response

In [16]:
# 用测试集的前10条，测试模型
val_path = "/home/code/chat_SQL/Dataset/GLM4/ENG/T2Q_table_Dataset/T2Q_GLM4_SFT_train_table.jsonl"
val_df = pd.read_json(val_path, lines=True)
test_df = val_df[:10]
print(test_df)
print(1)

                                         instruction  \
0   I want you to act as a relation extraction ro...   
1   I want you to act as a relation extraction ro...   
2   I want you to act as a relation extraction ro...   
3   I want you to act as a relation extraction ro...   
4   I want you to act as a relation extraction ro...   
5   I want you to act as a relation extraction ro...   
6   I want you to act as a relation extraction ro...   
7   I want you to act as a relation extraction ro...   
8   I want you to act as a relation extraction ro...   
9   I want you to act as a relation extraction ro...   

                                               input  \
0  Find the total revenue of companies of each fo...   
1         Which game type has least number of games?   
2  Return the description of the budget type that...   
3      What is the most cited paper of ohad shamir ?   
4                       How many students are there?   
5  Show the names and ages of editors and the t

In [17]:
import swanlab
test_text_list = []
for index, row in test_df.iterrows():
    instruction = row['instruction']
    input_value = row['input']
    
    messages = [
        {"role": "system", "content": f"{instruction}"},
        {"role": "user", "content": f"{input_value}"}
    ]

    response = predict(messages, model, tokenizer)
    print(input_value)
    print(response)
    messages.append({"role": "assistant", "content": f"{response}"})
    result_text = f"{messages[0]}\n\n{messages[1]}\n\n{messages[2]}"
#     test_text_list.append(swanlab.Text(result_text, caption=response))
    
# swanlab.log({"Prediction": test_text_list})
# swanlab.finish()

```Reference Table
-- Tables: manufacturers;
```
Find the total revenue of companies of each founder.
```Reference Table
-- Tables: manufacturers;
```
```Reference Table
-- Tables: video_games;
```
Which game type has least number of games?
```Reference Table
-- Tables: video_games;
```
```Reference Table
-- Tables: ref_budget_codes;
```
Return the description of the budget type that has the code ORG.
```Reference Table
-- Tables: ref_budget_codes;
```
```Reference Table
-- Tables: paper, cite, writes, author;
```
What is the most cited paper of ohad shamir ?
```Reference Table
-- Tables: paper, cite, writes, author;
```
```Reference Table
-- Tables: student;
```
How many students are there?
```Reference Table
-- Tables: student;
```
```Reference Table
-- Tables: editor, journal_committee, journal;
```
Show the names and ages of editors and the theme of journals for which they serve on committees, in ascending alphabetical order of theme.
```Reference Table
-- Tables: editor, journal_c